<div dir='rtl' style="font-family: Vazirmatn, IRANSans, 'Segoe UI', Tahoma, sans-serif; line-height: 1.9; font-size: 16px;">

## Byte-Pair Encoding (BPE)

BPE یا Byte-Pair Encoding ابتدا به‌عنوان روش فشرده‌سازی معرفی شد و بعدها در پردازش زبان طبیعی برای شکستن کلمات به زیرواحدها (subword) به‌کار رفت. ایده‌ی اصلی: به‌جای نگهداری هر کلمه به‌صورت کامل، واژگان را از قطعات پرتکرار می‌سازیم تا اندازه‌ی واژگان محدود و مسئله‌ی نشانه‌ی ناشناخته (UNK) کاهش یابد. این رویکرد برای زبان‌های با واژه‌های طولانی یا ترکیبی سودمند است.

### الگوریتم BPE (خلاصه‌ی گام‌به‌گام)
- **گام ۱: آماده‌سازی**: شکستن متن آموزشی به کلمات و سپس کاراکترها.
  - مثال: `low → l o w`، `lower → l o w e r`
- **گام ۲: شمارش جفت‌ها**: شمارش فراوانی تمام جفت‌های متوالی کاراکترها در کل پیکره.
- **گام ۳: ادغام پرتکرارترین جفت**: پرتکرارترین جفت را با یک توکن جدید جایگزین می‌کنیم.
  - مثال: اگر `l o` پرتکرارترین باشد: `l o w → lo w`، `l o w e r → lo w e r`
- **گام ۴: تکرار**: گام‌های ۲ و ۳ را تا رسیدن به اندازه‌ی واژگان هدف یا نبود جفت پرتکرار ادامه می‌دهیم.
- **گام ۵: ثبت قوانین ادغام**: تمام ادغام‌ها را به‌صورت «merge rules» ذخیره می‌کنیم.
- **گام ۶: توکنیزه‌سازی متن جدید**: اعمال ترتیبی قوانین ادغام برای شکستن واژه‌های جدید به subwordهای رایج.
  - نمونه: `text: lowest`، `merge rules: lo, we, st`، نتیجه: `lo w e st`

### مزایا
- **سادگی و سرعت آموزش**
- **کاهش UNK** (به‌ویژه با حالت مبتنی بر بایت)
- **مناسب برای زبان‌های با واژه‌های طولانی/ترکیبی**
- **واژگان کوچک‌تر نسبت به رویکرد واژه‌محور کامل**

### معایب
- **صرفاً مبتنی بر فراوانی**؛ معنای زبانی را لحاظ نمی‌کند.
- **احتمال تولید قطعات غیرمعنادار** برای واژه‌های نادر.
- **حساسیت به نویسه/فاصله**، برای زبان‌های با صرف پیچیده همیشه بهینه نیست.

### ابزارها و مدل‌های مرتبط
- **کتابخانه‌ها**: `subword-nmt`، Hugging Face `tokenizers`، `sentencepiece`
- **مدل‌ها**: GPT-2 و خانواده‌ی GPT، RoBERTa، BART، برخی سیستم‌های ترجمه مانند OpenNMT

</div>

<div dir='rtl' style="font-family: Vazirmatn, IRANSans, 'Segoe UI', Tahoma, sans-serif; line-height: 1.9; font-size: 16px;">

## WordPiece

WordPiece توسط گوگل معرفی شد؛ ابتدا در ترجمه‌ی ماشینی و سپس در BERT فراگیر شد. تفاوت کلیدی با BPE این است که افزودن زیرواحد جدید براساس بهبود «احتمال مدل زبانی» انجام می‌شود، نه صرفاً فراوانی وقوع.

### الگوریتم (خلاصه)
1. شروع از واژگان مبتنی بر کاراکتر.
2. ساخت کاندید برای افزودن به واژگان.
3. ارزیابی هر کاندید براساس بهبود احتمال متن تحت مدل زبانی.
4. افزودن بهترین کاندید به واژگان.
5. تکرار تا رسیدن به اندازه‌ی هدف واژگان.

### توکنیزه‌سازی متن جدید
- استفاده از راهبرد greedy longest-match.
- در BERT، زیرواحدهای درون‌کلمه‌ای با پیشوند `##` مشخص می‌شوند.
  - نمونه: `playing → play + ##ing`

### مزایا
- **انسجام معنایی بهتر** نسبت به BPE در سطح زیرواحد.
- **مناسب برای زبان‌های با فاصله‌گذاری مشخص** بین کلمات.

### معایب
- **پیچیدگی آموزشی بیشتر** از BPE.
- در نبود fallback ممکن است با واژه‌های جدید مشکل رخ دهد.

### ابزارها و مدل‌های مرتبط
- **کتابخانه‌ها**: Hugging Face `transformers`، Hugging Face `tokenizers`، TensorFlow Text
- **مدل‌ها**: BERT، mBERT، DistilBERT و دیگر مدل‌های گوگل

</div>

<div dir='rtl' style="font-family: Vazirmatn, IRANSans, 'Segoe UI', Tahoma, sans-serif; line-height: 1.9; font-size: 16px;">

## SentencePiece

SentencePiece توسط گوگل توسعه داده شد و بر خلاف بسیاری از روش‌ها، به جداسازی اولیه‌ی متن بر اساس فاصله نیاز ندارد؛ بنابراین می‌توان آن را مستقیماً روی متن خام به‌کار برد. این ویژگی برای زبان‌های بدون فاصله بین کلمات (مانند چینی و ژاپنی) و نیز مدل‌های چندزبانه بسیار مفید است.

SentencePiece دو حالت اصلی دارد:
1. **BPE mode**: مشابه الگوریتم BPE عمل می‌کند.
2. **Unigram mode**: مبتنی بر یک مدل احتمالاتی است.

### الگوریتم Unigram (خلاصه)
1. ساخت مجموعه‌ی بزرگی از زیرواحدهای کاندید.
2. تخصیص احتمال اولیه به هر زیرواحد.
3. به‌روزرسانی احتمال‌ها با EM برای بهینه‌سازی تقسیم‌بندی متن.
4. حذف واحدهای کم‌استفاده برای کوچک‌سازی واژگان.
5. انتخاب بهترین تقسیم‌بندی نهایی با الگوریتم Viterbi.

### ویژگی‌ها
- **بی‌نیاز از پیش‌جداسازی مبتنی بر فاصله**
- **کارآمد برای زبان‌های بدون فاصله**
- **امکان تولید تقسیم‌بندی‌های احتمالاتی متنوع**

### معایب
- **آموزش کندتر از BPE** در حالت Unigram.
- **نیاز به تنظیمات دقیق‌تر** در طول آموزش.

### نکته
- نشانه‌ی ویژه‌ی `▁` برای نمایش فاصله‌ی میان کلمات به‌کار می‌رود.

### ابزارها و مدل‌های مرتبط
- **کتابخانه‌ها**: `sentencepiece`، Hugging Face Transformers و Tokenizers
- **مدل‌ها**: T5، mT5، XLM-R، مدل‌های چندزبانه‌ی گوگل

</div>

In [6]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer,WordPieceTrainer

In [2]:
# Sample text
text = "This is a simple example to demonstrate BPE tokenization."

# Create a BPE tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Pre-tokenizer splits text into words with space
tokenizer.pre_tokenizer = Whitespace()

# Trainer defines how BPE merges subwords to build the vocabulary
trainer = BpeTrainer(vocab_size=50, min_frequency=1, special_tokens=["[UNK]"])

# Train the tokenizer on the sample text
tokenizer.train_from_iterator([text], trainer=trainer)

# Encode the text using the trained tokenizer
output = tokenizer.encode(text)

# Print the tokens, token IDs, and number of tokens
print("Tokens:", output.tokens)
print("Token IDs:", output.ids)
print("Number of tokens:", len(output.tokens))





Tokens: ['This', 'is', 'a', 'simple', 'example', 'to', 'demon', 'st', 'rat', 'e', 'BPE', 'token', 'ization', '.']
Token IDs: [46, 23, 6, 42, 48, 27, 47, 41, 40, 8, 45, 44, 49, 1]
Number of tokens: 14


In [7]:
#sample text
text = "This is a simple example to demonstrate WordPiece tokenization."

# Create a WordPiece tokenizer
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

# Pre-tokenizer splits text into words based on whitespace before applying WordPiece
tokenizer.pre_tokenizer = Whitespace()

# Trainer defines how WordPiece builds subwords vocabulary
trainer = WordPieceTrainer(vocab_size=50, min_frequency=1, special_tokens=["[UNK]"])

# Train the tokenizer on the sample text
tokenizer.train_from_iterator([text], trainer=trainer)

# Encode the text using the trained tokenizer
output = tokenizer.encode(text)

# Print the tokens, token IDs, and number of tokens
print("Tokens:", output.tokens)
print("Token IDs:", output.ids)
print("Number of tokens:", len(output.tokens))




Tokens: ['Th', '##i', '##s', 'i', '##s', 'a', 's', '##i', '##mple', 'ex', '##a', '##mple', 'to', 'de', '##m', '##on', '##s', '##t', '##r', '##at', '##e', 'Wo', '##r', '##d', '##P', '##i', '##e', '##c', '##e', 'to', '##k', '##e', '##n', '##i', '##z', '##at', '##i', '##on', '.']
Token IDs: [46, 27, 22, 10, 22, 5, 18, 27, 45, 49, 29, 45, 40, 48, 32, 41, 22, 30, 35, 42, 25, 47, 35, 36, 37, 27, 25, 38, 25, 40, 24, 25, 26, 27, 28, 42, 27, 41, 1]
Number of tokens: 39


In [8]:
import sentencepiece as spm

# Sample text for tokenization
text = "This is a simple example to demonstrate SentencePiece tokenization."

# First, we need to save the text to a temporary file for training
with open("sample.txt", "w", encoding="utf-8") as f:
    f.write(text)

# Train a SentencePiece model with a small vocabulary size for demonstration
spm.SentencePieceTrainer.train(
    input="sample.txt", 
    model_prefix="spm_model", 
    vocab_size=50, 
    model_type="bpe"  # can be 'bpe', 'unigram', 'char', or 'word'
)

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load("spm_model.model")

# Encode the text
tokens = sp.encode(text, out_type=str)
token_ids = sp.encode(text, out_type=int)

# Print tokens and IDs
print("Tokens:", tokens)
print("Token IDs:", token_ids)
print("Number of tokens:", len(tokens))

Tokens: ['▁', 'Th', 'is', '▁', 'is', '▁a', '▁', 'si', 'mple', '▁', 'exa', 'mple', '▁to', '▁', 'dem', 'on', 'st', 'r', 'ate', '▁S', 'ent', 'en', 'ce', 'Pie', 'ce', '▁to', 'k', 'en', 'iz', 'ati', 'on', '.']
Token IDs: [29, 14, 6, 29, 6, 21, 29, 18, 12, 29, 27, 12, 11, 29, 25, 9, 19, 47, 23, 20, 26, 3, 5, 22, 5, 11, 46, 3, 17, 24, 9, 40]
Number of tokens: 32


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: sample.txt
  input_format: 
  model_prefix: spm_model
  model_type: BPE
  vocab_size: 50
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential